# TP Implémentation du modèle linéaire L1 régularisé


Puisqu'il y a du code fourni pour une application similaire je commence par l'éxécuter pour voir ce qu'il fait et ce qui sera réutilisable.

In [178]:
import numpy as np
from sklearn.base import BaseEstimator,ClassifierMixin
from sklearn.svm import SVC
from sklearn import cross_validation
class RandomOuSvmClassifier(BaseEstimator,ClassifierMixin):
    """ si la nature du classifier est random il predit au hasard des 0 et
    des 1 sinon il utilise un svm pour predire"""
    def __init__(self,nature="random"):
        self.nature=nature
        self.svm=SVC()
    def fit(self, X, y):
        if(self.nature=="svm"):
            self.svm.fit(X, y)
        return self
    def predict(self, X):
        if(self.nature=="random"):
            return np.random.randint(0,2,len(X))
        else:
            return self.svm.predict(X)

C'est visiblement l'implémentation d'un classifieur avec la même interface que ceux de sklearn, et donc la même que celle du classifieur demandé
je jette un oeil aux docs de http://scikit-learn.org/stable/modules/generated/sklearn.base.BaseEstimator.html et http://scikit-learn.org/stable/modules/generated/sklearn.base.ClassifierMixin.html pour voir s'il est nécessaire d'en hériter.(la doc dit que oui mais je ne comprends pas trop pourquoi...)

In [179]:
from sklearn import datasets
from sklearn import metrics
iris = datasets.load_iris()
X = iris.data
y = iris.target
classifieurRandom= RandomOuSvmClassifier(nature="random")
classifieurSvm= RandomOuSvmClassifier(nature="svm")
scoresRandom = cross_validation.cross_val_score(classifieurRandom, X, y, cv=5
,scoring="accuracy")
scoresSvm = cross_validation.cross_val_score(classifieurSvm, X, y, cv=5
,scoring="accuracy")

In [182]:
scoresRandom

array([ 0.33333333,  0.4       ,  0.26666667,  0.26666667,  0.33333333])

In [183]:
scoresSvm

array([ 0.96666667,  1.        ,  0.96666667,  0.96666667,  1.        ])

Ce deuxième bout de code m'a donc montré comment charger un ensemble de données qui possède 3 classes (au vu du random effectué dans le predict et de ses résultats) Il faudra donc en utiliser un autre car notre classifications se fait sur 2 classes. Il me montre aussi comment utiliser la cross-validation.

In [72]:
import numpy as np
from sklearn.base import BaseEstimator,ClassifierMixin
from sklearn.svm import SVC
from sklearn import cross_validation
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from numpy.linalg import norm

class L1LinearRegularisedClassifier(BaseEstimator,ClassifierMixin):
   
    def __init__(self,regularization_weightL1 = 0,regularization_weightL2 = 0, step = 0.00003 ,iterations = 5000):       
        
        self.regularization_weight = regularization_weightL1 #possibilité de faire une régularisation L1
        self.regularization_weight = regularization_weightL2 #possibilité de faire une régularisation L2
       
        self.step = step # pas du gradient
        self.theta = np.array(np.random.rand(X.shape[1]+1),dtype='float64') # theta initial au hasard
        self.iterations = iterations # nombre d'itérations
    
    def regularizationL1(self,theta):
        return self.regularization_weightL1 * norm(theta,1)
    def regularizationL2(self,theta):
        return self.regularization_weightL2 * norm(theta,2)
    
    def fit(self, X, y):
        
        self.stochastic_gradient_descent_L1(X,y,self.theta,self.iterations,self.step)
        #print " Ready to predict"
        #print self.theta
        return self
    
    def stochastic_gradient_descent_L1(self,X,y,theta,I,step):
        #print theta.shape
        #print X.shape
        X_changed = np.concatenate((np.ones((X.shape[0],1),dtype='float64'),X),axis=1)
        for it in range(I):
            idx = np.random.randint(y.size)
            grad_cost = np.zeros(X_changed.shape[1])
            prod = np.dot(X_changed[idx],self.theta)
            grad_cost = 2.0 * (prod-y[idx])*X_changed[idx]
            theta_prime = self.theta - step*grad_cost 
            if self.regularization_weight > 0:
                theta_prime = theta_prime + self.regularization_weight * np.sign(self.theta)
                for i in range(self.theta.size):
                    if self.theta[i]*theta_prime[i] > 0 :
                        self.theta[i] = 0
                    else :
                        self.theta[i] = theta_prime[i]
            else : 
                self.theta = theta_prime
            if it % 5000 == 0:
                y_pred = self.predict(X)
                #print it
                #print self.cost_function(y,y_pred,mean_squared_error,self.regularizationL1), "coût"
                #print accuracy_score(y,y_pred), "accuracy"
    
    def cost_function(self,y,y_pred,error_function,regularization_function1,regularization_function2) :
        cost = error_function(y,y_pred)-regularization_function1(self.theta) - regularization_function1(self.theta)
        return cost
    
    def predict(self, X):
        y_pred = np.zeros(X.shape[0])
        X_changed = np.concatenate((np.ones((X.shape[0],1),dtype='float64'),X),axis=1) 
        for i in range(X_changed.shape[0]):
            y_pred[i] = np.dot(X_changed[i],self.theta)
            if y_pred[i] > 0:
                y_pred[i] = 1
            else:
                y_pred[i] = -1
        
        return y_pred

from sklearn import datasets
from sklearn import metrics
from sklearn.datasets import fetch_mldata
digits = datasets.load_digits(n_class = 2)
X = digits.data
y = digits.target
y = 2*y - 1
print type(X[0][0])
print type(y[0])
print y[:10]
classifieurL1= L1LinearRegularisedClassifier()
scoresL1 = cross_validation.cross_val_score(classifieurL1, X, y, cv=5,scoring="accuracy"
)
scoresL1
np.mean(scoresL1)

<type 'numpy.float64'>
<type 'numpy.int32'>
[-1  1 -1  1 -1  1 -1 -1  1  1]


0.75814575428216169

In [ ]:
import matplotlib.pyplot as plt
nb_lambdas = 100
a = np.zeros(nb_lambdas)
reg = np.zeros(nb_lambdas)
reg[0] = 0

for i in range(1,nb_lambdas):
    reg[i] = 2**(i-nb_lambda)
    

for i in range(0,nb_lambdas):

    classifieurL1= L1LinearRegularisedClassifier(regularization_weight = reg[i])
    scoresL1 = cross_validation.cross_val_score(classifieurL1, X, y, cv=5,scoring="accuracy")
    a[i] = np.mean(scoresL1)

In [73]:
import matplotlib.pyplot as plt
nb_lambdas = 100
a = np.zeros(nb_lambdas)
reg = np.zeros(nb_lambdas)
reg[0] = 0

for i in range(1,nb_lambdas):
    reg[i] = 10**(-1000*i)
    

for i in range(0,nb_lambdas):

    classifieurL1= L1LinearRegularisedClassifier(regularization_weight = reg[i])
    scoresL1 = cross_validation.cross_val_score(classifieurL1, X, y, cv=5,scoring="accuracy")
    a[i] = np.mean(scoresL1)



In [77]:
a   
plt.plot(a)
plt.ylabel('accuracy')
plt.xlabel('ln_lambda')
plt.show()

In [74]:
reg



array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [75]:
a

array([ 0.76111379,  0.73279257,  0.83543315,  0.70840461,  0.62826334,
        0.727975  ,  0.75590392,  0.79445034,  0.6781122 ,  0.7275827 ,
        0.7279675 ,  0.79707591,  0.75077122,  0.79506989,  0.76728889,
        0.73865683,  0.76609535,  0.69729136,  0.74162594,  0.70601378,
        0.73224698,  0.77570047,  0.75208802,  0.78337514,  0.79425794,
        0.77778474,  0.71921238,  0.75617242,  0.77801413,  0.73626493,
        0.76084421,  0.74949729,  0.70010933,  0.76088334,  0.69420702,
        0.78356968,  0.76898514,  0.75806215,  0.77735546,  0.76203454,
        0.73078655,  0.67213272,  0.74409501,  0.78024739,  0.74695104,
        0.76682798,  0.77743906,  0.79144211,  0.74448839,  0.76930028,
        0.76393284,  0.76115291,  0.78071795,  0.75544301,  0.74618143,
        0.74718685,  0.68876455,  0.70825026,  0.76416222,  0.75814361,
        0.73911559,  0.7302833 ,  0.72723701,  0.74668146,  0.74235535,
        0.70790244,  0.71345371,  0.77226403,  0.78025168,  0.72

In [79]:

nb_lambdas = 100
a2 = np.zeros(nb_lambdas)
reg2 = np.zeros(nb_lambdas)
reg2[0] = 0

for i in range(1,nb_lambdas):
    reg2[i] = 10**(-1000) +  2*10**(-100000*i)
    

for i in range(0,nb_lambdas):

    classifieurL1= L1LinearRegularisedClassifier(regularization_weight = reg2[i])
    scoresL1 = cross_validation.cross_val_score(classifieurL1, X, y, cv=5,scoring="accuracy")
    a2[i] = np.mean(scoresL1)

http://www.aclweb.org/anthology/P09-1054

In [80]:
a2

array([ 0.77797286,  0.74417111,  0.77458304,  0.80004931,  0.76628561,
        0.7395079 ,  0.74765097,  0.72657942,  0.74108892,  0.78665831,
        0.76624756,  0.82242374,  0.78345231,  0.70285013,  0.76068986,
        0.77173023,  0.7771652 ,  0.74231837,  0.78901108,  0.77168575,
        0.73348608,  0.79144104,  0.72576586,  0.74772601,  0.72194782,
        0.75860238,  0.74783909,  0.73903949,  0.7582165 ,  0.75239565,
        0.75277831,  0.78600071,  0.75003644,  0.67834373,  0.71608142,
        0.79402963,  0.71986462,  0.783224  ,  0.77770971,  0.77546788,
        0.76427959,  0.73873507,  0.70578226,  0.73336549,  0.73005392,
        0.74378845,  0.72167931,  0.78773608,  0.74251935,  0.7279675 ,
        0.72815776,  0.72773276,  0.7585654 ,  0.74714344,  0.72762718,
        0.76095408,  0.74119986,  0.78036798,  0.78078976,  0.74722276,
        0.7527826 ,  0.72688812,  0.72731312,  0.74741945,  0.76142142,
        0.7244169 ,  0.71145304,  0.78495938,  0.74104444,  0.72

In [ ]:
   
plt.plot(a2)
plt.ylabel('accuracy')
plt.xlabel('ln_lambda')
plt.show()

In [39]:
Y.shape

(5L,)

In [40]:
Y.size


5

In [69]:
A = np.random.rand(6,5)

In [55]:
A

array([[ 0.61990672,  0.85698778,  0.30227644,  0.33878824,  0.99416707],
       [ 0.62365261,  0.58393254,  0.54403266,  0.3566358 ,  0.09184679],
       [ 0.85410289,  0.58495266,  0.94803139,  0.03989068,  0.29810894],
       [ 0.74813547,  0.85936373,  0.35325312,  0.02635477,  0.24077968],
       [ 0.18114573,  0.57853052,  0.06748894,  0.09179829,  0.12952997]])

In [61]:
sum(A[1])

2.2001003949167433

In [63]:
8%9


8

In [152]:
mean_squared_error(np.ones(5),np.zeros(5))

1.0